In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

TMDB API에서 영화id를 이용하여 각 영화의 cast정보를 가져올 수 있음. 

"https://api.themoviedb.org/3/movie/{movie_id}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"

In [2]:
# 영화id가 512195인 cast정보 크롤링

cast_url="https://api.themoviedb.org/3/movie/512195?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
response = requests.get(cast_url)
json_object = response.content
json.loads(json_object)

# 항목 credits아래 cast정보가 있음. 'credits': {'cast': [{'adult': False,....

{'adult': False,
 'backdrop_path': '/dK12GIdhGP6NPGFssK2Fh265jyr.jpg',
 'belongs_to_collection': None,
 'budget': 200000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 53, 'name': 'Thriller'}],
 'homepage': 'https://www.netflix.com/us/title/81161626',
 'id': 512195,
 'imdb_id': 'tt7991608',
 'original_language': 'en',
 'original_title': 'Red Notice',
 'overview': "An Interpol-issued Red Notice is a global alert to hunt and capture the world's most wanted. But when a daring heist brings together the FBI's top profiler and two rival criminals, there's no telling what will happen.",
 'popularity': 3991.047,
 'poster_path': '/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg',
 'production_companies': [{'id': 34081,
   'logo_path': None,
   'name': 'Flynn Picture Company',
   'origin_country': 'US'},
  {'id': 73669,
   'logo_path': '/7tmSGstK3KwgcDIuBYLTAayjit9.png',
   'name': 'Seven Bucks Productions',
   'origin_country': 'US'},
  {'i

In [3]:
# cast 정보만 가져오기
movie_data=json.loads(json_object)
movie_data["credits"]["cast"]

[{'adult': False,
  'gender': 2,
  'id': 18918,
  'known_for_department': 'Acting',
  'name': 'Dwayne Johnson',
  'original_name': 'Dwayne Johnson',
  'popularity': 33.478,
  'profile_path': '/kuqFzlYMc2IrsOyPznMd1FroeGq.jpg',
  'cast_id': 2,
  'character': 'John Hartley',
  'credit_id': '5aac343892514116b7004626',
  'order': 0},
 {'adult': False,
  'gender': 2,
  'id': 10859,
  'known_for_department': 'Acting',
  'name': 'Ryan Reynolds',
  'original_name': 'Ryan Reynolds',
  'popularity': 43.06,
  'profile_path': '/4SYTH5FdB0dAORV98Nwg3llgVnY.jpg',
  'cast_id': 13,
  'character': 'Nolan Booth',
  'credit_id': '5d23cf7069d2800010011045',
  'order': 1},
 {'adult': False,
  'gender': 1,
  'id': 90633,
  'known_for_department': 'Acting',
  'name': 'Gal Gadot',
  'original_name': 'Gal Gadot',
  'popularity': 29.252,
  'profile_path': '/fysvehTvU6bE3JgxaOTRfvQJzJ4.jpg',
  'cast_id': 9,
  'character': 'Sarah Black / The Bishop',
  'credit_id': '5b1ea4290e0a261010008a2d',
  'order': 2},
 {'ad

In [4]:
# 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv',index_col=0) 

C:\Users\user\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
# 영화id를 이용하여 cast 정보 crawling  (너무 오래걸림)

tmdb_movie_id_list = tmdb_movie_list[["id"]]

all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):  # movie_id 목록을 기준으로 인덱스와 movie_id를 반환
    try :
        cast_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(cast_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        cast_list = movie_data["credits"]["cast"]
        for cast in cast_list:                                      
            cast["movie_id"] = movie_id
            all_cast_list.append(cast)

    except:
        error_list.append(movie_id)

    if idx == 100:
        break

In [10]:
# cast 정보 csv파일로 저장 
casts=pd.DataFrame(all_cast_list)
casts


,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,2,54768,Acting,Turo Pajala,Turo Pajala,1.219,/b6JdzqTn6UFYf4DouHbbE8Ypk4r.jpg,3,Taisto Olavi Kasurinen,52fe420dc3a36847f8000029,0,2.0
1,False,1,54769,Acting,Susanna Haavisto,Susanna Haavisto,1.400,/lhjoqlBFCvshchMAd2lQLWV7z7I.jpg,4,Irmeli Katariina Pihlaja,52fe420dc3a36847f800002d,1,2.0
2,False,2,4826,Acting,Matti Pellonpää,Matti Pellonpää,2.112,/1Qzhkkp3wFE2NMGKVyOra9931q2.jpg,5,Mikkonen,52fe420dc3a36847f8000031,2,2.0
3,False,2,54770,Acting,Eetu Hilkamo,Eetu Hilkamo,0.600,None,6,Riku,52fe420dc3a36847f8000035,3,2.0
4,False,2,1177496,Acting,Erkki Pajala,Erkki Pajala,0.600,/lJ7GmiHT1UPxmURqQCLT4L5PIMa.jpg,12,Miner,5e1cda97d5dbc20017f439bb,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,False,0,1486715,Acting,Cornelia Thaw,Cornelia Thaw,0.980,/jEnEo8pVCOQNR5p9JyqL87Wvq8Q.jpg,36,Dracula's Bride (uncredited),55a31c99c3a3681cf500454c,21,138.0
3635,False,1,96141,Acting,Dorothy Tree,Dorothy Tree,0.662,/i3SHsZRr0STkrMBL58TCMctXdz7.jpg,37,Dracula's Bride (uncredited),55a31ce69251412979004019,22,138.0
3636,False,0,1486716,Acting,Josephine Velez,Josephine Velez,0.600,None,38,Nurse Grace (uncredited),55a31cf89251412979004020,23,138.0
3637,False,2,120755,Acting,Michael Visaroff,Michael Visaroff,0.997,/dHhWKf8vp24ZWwUvOLQfLIJOODk.jpg,39,Innkeeper (uncredited),55a31d04c3a3681cec003efc,24,138.0


In [ ]:
casts.to_csv("../data-files/cast.csv", header=True, index=True, mode='w')

In [7]:
# cast정보 DB(mysql)에 저장 
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

#cursor.execute('DELETE FROM casts')

with conn.cursor() as cursor: # with가 종료될 때 cursor.close() 자동 호출
    cursor.execute()
    cursor.execute()
    conn.commit() # 이전에 실행된 SQL 결과를 확정

conn.close()

NameError: name 'cursor' is not defined